In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import feature_selection
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

# Prepare the dataset to apply different models

In [2]:
df = pd.read_excel('Merged_table_from_2005.xlsx')
df

,Year,Country,Date,Competition,Home_team,Away_team,Status,Result,1x2,number_of_games_last_days_home_team,...,Home_team_OVR,Home_team_Rival_team,Home_team_Budget_Mill_€,Away_team_ATT,Away_team_MID,Away_team_DEF,Away_team_OVR,Away_team_Rival_team,Away_team_Budget_Mill_€,Rivals
0,2005,Portugal,22 May 05,National League,Boavista,Benfica,Finalized,1 - 1,x,2,...,58,Vitória SC,0.375,80,74,74,75,Sporting Lisbon,5.0,0
1,2005,Portugal,07 May 05,National League,Penafiel,Benfica,Finalized,1 - 0,1,3,...,61,FC Porto,0.100,80,74,74,75,Sporting Lisbon,5.0,0
2,2005,Portugal,24 Apr 05,National League,Estoril,Benfica,Finalized,1 - 2,2,3,...,59,Vitória Futebol Clube,2.000,80,74,74,75,Sporting Lisbon,5.0,0
3,2005,Portugal,10 Apr 05,National League,Rio Ave,Benfica,Finalized,1 - 0,1,2,...,52,Benfica,2.900,80,74,74,75,Sporting Lisbon,5.0,1
4,2005,Portugal,19 Mar 05,National League,Vitória Setúbal,Benfica,Finalized,0 - 2,2,4,...,61,Belenenses,0.350,80,74,74,75,Sporting Lisbon,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37745,2022,France,03 Oct 21,National League,Stade Rennais,PSG,Finalized,2 - 0,1,4,...,76,FC Nantes,14.000,88,83,85,86,Olympique de Marseille,160.0,0
37746,2022,France,22 Sep 21,National League,Metz,PSG,Finalized,1 - 2,2,2,...,73,AS Nancy,7.000,88,83,85,86,Olympique de Marseille,160.0,0
37747,2022,France,29 Aug 21,National League,Stade de Reims,PSG,Finalized,0 - 2,2,3,...,73,ESTAC Troyes,8.500,88,83,85,86,Olympique de Marseille,160.0,0
37748,2022,France,20 Aug 21,National League,Stade Brestois,PSG,Finalized,2 - 4,2,2,...,73,En Avant Guingamp,6.500,88,83,85,86,Olympique de Marseille,160.0,0


In [4]:
list(df.columns)

['Year',
 'Country',
 'Date',
 'Competition',
 'Home_team',
 'Away_team',
 'Status',
 'Result',
 '1x2',
 'number_of_games_last_days_home_team',
 'number_of_games_last_days_away_team',
 'points_respective_year_home_team',
 'points_respective_year_away_team',
 'points_last_games_home_team',
 'points_last_games_away_team',
 'points_between_teams_home_team',
 'points_between_teams_away_team',
 'Home_team_ATT',
 'Home_team_MID',
 'Home_team_DEF',
 'Home_team_OVR',
 'Home_team_Rival_team',
 'Home_team_Budget_Mill_€',
 'Away_team_ATT',
 'Away_team_MID',
 'Away_team_DEF',
 'Away_team_OVR',
 'Away_team_Rival_team',
 'Away_team_Budget_Mill_€',
 'Rivals']

In [5]:
# We can drop the columns that do not add value to the results prediction

df = df.drop(['Date','Competition','Result','Home_team_Rival_team',
             'Away_team_Rival_team'], axis=1)

df

,Year,Country,Home_team,Away_team,Status,1x2,number_of_games_last_days_home_team,number_of_games_last_days_away_team,points_respective_year_home_team,points_respective_year_away_team,...,Home_team_MID,Home_team_DEF,Home_team_OVR,Home_team_Budget_Mill_€,Away_team_ATT,Away_team_MID,Away_team_DEF,Away_team_OVR,Away_team_Budget_Mill_€,Rivals
0,2005,Portugal,Boavista,Benfica,Finalized,x,2,2,49,64,...,64,53,58,0.375,80,74,74,75,5.0,0
1,2005,Portugal,Penafiel,Benfica,Finalized,1,3,4,37,61,...,67,55,61,0.100,80,74,74,75,5.0,0
2,2005,Portugal,Estoril,Benfica,Finalized,2,3,4,26,55,...,60,52,59,2.000,80,74,74,75,5.0,0
3,2005,Portugal,Rio Ave,Benfica,Finalized,1,2,1,39,54,...,54,48,52,2.900,80,74,74,75,5.0,1
4,2005,Portugal,Vitória Setúbal,Benfica,Finalized,2,4,4,32,48,...,60,56,61,0.350,80,74,74,75,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37745,2022,France,Stade Rennais,PSG,Finalized,1,4,5,9,24,...,76,75,76,14.000,88,83,85,86,160.0,0
37746,2022,France,Metz,PSG,Finalized,2,2,3,3,18,...,72,73,73,7.000,88,83,85,86,160.0,0
37747,2022,France,Stade de Reims,PSG,Finalized,2,3,2,3,9,...,72,74,73,8.500,88,83,85,86,160.0,0
37748,2022,France,Stade Brestois,PSG,Finalized,2,2,3,2,6,...,74,73,73,6.500,88,83,85,86,160.0,0


In [6]:
# Let's check the correlation between variables

corr = df.corr()
corr.style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1).set_precision(2)

,Year,number_of_games_last_days_home_team,number_of_games_last_days_away_team,points_respective_year_home_team,points_respective_year_away_team,points_last_games_home_team,points_last_games_away_team,points_between_teams_home_team,points_between_teams_away_team,Home_team_ATT,Home_team_MID,Home_team_DEF,Home_team_OVR,Home_team_Budget_Mill_€,Away_team_ATT,Away_team_MID,Away_team_DEF,Away_team_OVR,Away_team_Budget_Mill_€,Rivals
Year,1.00,-0.02,-0.02,-0.02,-0.02,-0.05,-0.07,0.47,0.44,0.06,0.21,0.22,0.22,0.20,0.06,0.21,0.22,0.22,0.20,0.00
number_of_games_last_days_home_team,-0.02,1.00,0.61,0.34,0.22,0.31,0.18,0.10,-0.02,0.26,0.26,0.26,0.27,0.24,0.05,0.05,0.05,0.05,0.03,0.02
number_of_games_last_days_away_team,-0.02,0.61,1.00,0.23,0.34,0.17,0.31,-0.00,0.12,0.06,0.05,0.05,0.05,0.04,0.25,0.25,0.25,0.26,0.23,0.02
points_respective_year_home_team,-0.02,0.34,0.23,1.00,0.69,0.51,0.21,0.15,-0.01,0.29,0.30,0.30,0.31,0.28,0.02,0.02,0.02,0.02,0.01,0.03
points_respective_year_away_team,-0.02,0.22,0.34,0.69,1.00,0.20,0.51,0.01,0.17,0.02,0.02,0.02,0.02,0.01,0.29,0.30,0.30,0.31,0.28,0.02
points_last_games_home_team,-0.05,0.31,0.17,0.51,0.20,1.00,0.27,0.16,-0.02,0.32,0.33,0.32,0.34,0.31,0.00,-0.00,0.00,-0.00,-0.00,0.03
points_last_games_away_team,-0.07,0.18,0.31,0.21,0.51,0.27,1.00,0.01,0.18,0.01,0.00,0.00,0.01,-0.00,0.32,0.33,0.32,0.33,0.30,0.03
points_between_teams_home_team,0.47,0.10,-0.00,0.15,0.01,0.16,0.01,1.00,0.40,0.31,0.38,0.39,0.40,0.39,0.09,0.16,0.17,0.17,0.10,0.05
points_between_teams_away_team,0.44,-0.02,0.12,-0.01,0.17,-0.02,0.18,0.40,1.00,0.05,0.11,0.12,0.12,0.05,0.33,0.39,0.40,0.41,0.42,0.05
Home_team_ATT,0.06,0.26,0.06,0.29,0.02,0.32,0.01,0.31,0.05,1.00,0.84,0.84,0.90,0.63,0.27,0.28,0.29,0.29,0.14,0.04


In [7]:
# There's a high correlation between Home_team_OVR and the other columns with
# ratings (Home_team_ATT, Home_team_MID, Home_team_DEF). Also, the column 
# Home_team_Budget_Mill_€ is highly correlated. Let's drop these columns,
# together with the respective ones of the away teams.

df = df.drop(['Home_team_ATT','Home_team_MID','Home_team_DEF','Home_team_Budget_Mill_€',
 'Away_team_ATT','Away_team_MID','Away_team_DEF','Away_team_Budget_Mill_€',], axis=1)

df

,Year,Country,Home_team,Away_team,Status,1x2,number_of_games_last_days_home_team,number_of_games_last_days_away_team,points_respective_year_home_team,points_respective_year_away_team,points_last_games_home_team,points_last_games_away_team,points_between_teams_home_team,points_between_teams_away_team,Home_team_OVR,Away_team_OVR,Rivals
0,2005,Portugal,Boavista,Benfica,Finalized,x,2,2,49,64,1,10,0,0,58,75,0
1,2005,Portugal,Penafiel,Benfica,Finalized,1,3,4,37,61,9,10,0,0,61,75,0
2,2005,Portugal,Estoril,Benfica,Finalized,2,3,4,26,55,3,10,0,0,59,75,0
3,2005,Portugal,Rio Ave,Benfica,Finalized,1,2,1,39,54,7,13,0,0,52,75,1
4,2005,Portugal,Vitória Setúbal,Benfica,Finalized,2,4,4,32,48,3,11,0,0,61,75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37745,2022,France,Stade Rennais,PSG,Finalized,1,4,5,9,24,4,15,25,22,76,86,0
37746,2022,France,Metz,PSG,Finalized,2,2,3,3,18,2,15,7,16,73,86,0
37747,2022,France,Stade de Reims,PSG,Finalized,2,3,2,3,9,3,9,8,8,73,86,0
37748,2022,France,Stade Brestois,PSG,Finalized,2,2,3,2,6,2,6,1,13,73,86,0


In [8]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1).set_precision(2)

,Year,number_of_games_last_days_home_team,number_of_games_last_days_away_team,points_respective_year_home_team,points_respective_year_away_team,points_last_games_home_team,points_last_games_away_team,points_between_teams_home_team,points_between_teams_away_team,Home_team_OVR,Away_team_OVR,Rivals
Year,1.00,-0.02,-0.02,-0.02,-0.02,-0.05,-0.07,0.47,0.44,0.22,0.22,0.00
number_of_games_last_days_home_team,-0.02,1.00,0.61,0.34,0.22,0.31,0.18,0.10,-0.02,0.27,0.05,0.02
number_of_games_last_days_away_team,-0.02,0.61,1.00,0.23,0.34,0.17,0.31,-0.00,0.12,0.05,0.26,0.02
points_respective_year_home_team,-0.02,0.34,0.23,1.00,0.69,0.51,0.21,0.15,-0.01,0.31,0.02,0.03
points_respective_year_away_team,-0.02,0.22,0.34,0.69,1.00,0.20,0.51,0.01,0.17,0.02,0.31,0.02
points_last_games_home_team,-0.05,0.31,0.17,0.51,0.20,1.00,0.27,0.16,-0.02,0.34,-0.00,0.03
points_last_games_away_team,-0.07,0.18,0.31,0.21,0.51,0.27,1.00,0.01,0.18,0.01,0.33,0.03
points_between_teams_home_team,0.47,0.10,-0.00,0.15,0.01,0.16,0.01,1.00,0.40,0.40,0.17,0.05
points_between_teams_away_team,0.44,-0.02,0.12,-0.01,0.17,-0.02,0.18,0.40,1.00,0.12,0.41,0.05
Home_team_OVR,0.22,0.27,0.05,0.31,0.02,0.34,0.01,0.40,0.12,1.00,0.35,0.04


In [9]:
# Let's define X and y. We should exclude the column "Status", as this is
# only for information and thus should not be used to train the algorithms.

X = df.drop(['Status','1x2'], axis=1)
y = df['1x2']

In [10]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=54)

In [11]:
X_train.dtypes

Year                                    int64
Country                                object
Home_team                              object
Away_team                              object
number_of_games_last_days_home_team     int64
number_of_games_last_days_away_team     int64
points_respective_year_home_team        int64
points_respective_year_away_team        int64
points_last_games_home_team             int64
points_last_games_away_team             int64
points_between_teams_home_team          int64
points_between_teams_away_team          int64
Home_team_OVR                           int64
Away_team_OVR                           int64
Rivals                                  int64
dtype: object

In [12]:
# Although the columns "Year" and "Rivals" are of type integer, they are basically
# "tags" and should be considered as text, because its values being higher or lower
# should not be considered better or worse, unlike the other columns of type integer.

X_train_numeric_data = X_train.select_dtypes(include=['int64', 'float64']
                ).drop(['Year','Rivals'],axis=1)
X_test_numeric_data = X_test.select_dtypes(include=['int64', 'float64']
                ).drop(['Year','Rivals'],axis=1)

X_train_categorical_data = X_train[list(X_train.select_dtypes(include=['object']).columns) +\
                                      ['Year','Rivals']]

X_test_categorical_data = X_test[list(X_test.select_dtypes(include=['object']).columns) +\
                                      ['Year','Rivals']]

In [13]:
X_train_numeric_data.dtypes

number_of_games_last_days_home_team    int64
number_of_games_last_days_away_team    int64
points_respective_year_home_team       int64
points_respective_year_away_team       int64
points_last_games_home_team            int64
points_last_games_away_team            int64
points_between_teams_home_team         int64
points_between_teams_away_team         int64
Home_team_OVR                          int64
Away_team_OVR                          int64
dtype: object

In [14]:
X_train_categorical_data.dtypes

Country      object
Home_team    object
Away_team    object
Year          int64
Rivals        int64
dtype: object

In [15]:
numeric_columns = X.select_dtypes(include=['int64', 'float64']).drop(['Year','Rivals'],axis=1).columns

categorical_columns = X[list(X.select_dtypes(include=['object']).columns) +\
                                      ['Year','Rivals']].columns

In [16]:
numeric_columns

Index(['number_of_games_last_days_home_team',
       'number_of_games_last_days_away_team',
       'points_respective_year_home_team', 'points_respective_year_away_team',
       'points_last_games_home_team', 'points_last_games_away_team',
       'points_between_teams_home_team', 'points_between_teams_away_team',
       'Home_team_OVR', 'Away_team_OVR'],
      dtype='object')

In [17]:
categorical_columns

Index(['Country', 'Home_team', 'Away_team', 'Year', 'Rivals'], dtype='object')

# Training and evaluation of the models

In [18]:
warnings.filterwarnings("ignore")

In [23]:
%%time

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)])



dumb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DummyClassifier(strategy='most_frequent'))])

kn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsClassifier())])

linear_svc = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', svm.LinearSVC())])

dt = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeClassifier(random_state=0))])

lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())])

rfc = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier())])

models = [dumb,kn,linear_svc,dt,lr,rfc]

models_readable_list = ['dumb','kn','linear_svc','dt','lr','rfc']

metrics = ['Weighted F1 mean - no tuning',
                'Weighted F1 std - no tuning', 'Weighted F1 scores - no tuning'
]

d = {}

for metric in metrics:
    d[metric] = []

for index,model in enumerate(models):
    
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_weighted')
    
    rounded_scores = []
    for score in scores:
        rounded_scores.append("{:.4f}".format(score))
    
    
    d['Weighted F1 mean - no tuning'].append("{:.4f}".format(
        scores.mean()))
    
    d['Weighted F1 std - no tuning'].append("{:.4f}".format(
        scores.std()))
    
    d['Weighted F1 scores - no tuning'].append(rounded_scores)


models_evaluation = pd.DataFrame(d, index = models_readable_list)
models_evaluation

CPU times: user 4min 3s, sys: 8.17 s, total: 4min 11s
Wall time: 4min 12s


,Weighted F1 mean - no tuning,Weighted F1 std - no tuning,Weighted F1 scores - no tuning
dumb,0.2696,0.0001,"[0.2695, 0.2697, 0.2698, 0.2698, 0.2696]"
kn,0.4401,0.0027,"[0.4387, 0.4372, 0.4429, 0.4379, 0.4438]"
linear_svc,0.4661,0.0037,"[0.4600, 0.4642, 0.4708, 0.4671, 0.4684]"
dt,0.4334,0.0026,"[0.4292, 0.4317, 0.4341, 0.4365, 0.4354]"
lr,0.4716,0.0048,"[0.4623, 0.4713, 0.4749, 0.4749, 0.4746]"
rfc,0.4625,0.0064,"[0.4583, 0.4525, 0.4656, 0.4652, 0.4711]"


# Optimizing models parameters

### K-neighbours model

In [24]:
%%time

model = kn

param_grid = {
    'model__n_neighbors': [1,5,10,15,20],
    'model__metric': ['euclidean', 'manhattan', 'minkowski'],
    'model__weights':['uniform', 'distance']
}

CV_kn = GridSearchCV(
    model,
    param_grid,
    scoring='f1_weighted',
    n_jobs= -1,
    cv=5
)
                  
CV_kn.fit(X_train, y_train)  
print(CV_kn.best_params_)    
print(CV_kn.best_score_)

{'model__metric': 'manhattan', 'model__n_neighbors': 20, 'model__weights': 'distance'}
0.4645017897518257
CPU times: user 2.07 s, sys: 472 ms, total: 2.54 s
Wall time: 10min 36s


### Linear SVC

In [25]:
%%time

model = linear_svc

param_grid = {
    'model__C': [10,1,0.1],
    'model__penalty': ['l1', 'l2'],
    'model__multi_class': ['ovr', 'crammer_singer']
}

CV_linear_svc = GridSearchCV(
    model,
    param_grid,
    scoring='f1_weighted',
    n_jobs= -1,
    cv=5
)
                  
CV_linear_svc.fit(X_train, y_train)  
print(CV_linear_svc.best_params_)    
print(CV_linear_svc.best_score_)

{'model__C': 10, 'model__multi_class': 'ovr', 'model__penalty': 'l2'}
0.46779471893754804
CPU times: user 21.8 s, sys: 361 ms, total: 22.2 s
Wall time: 10min 28s


### Decision Tree model

In [26]:
%%time

model = dt

param_grid = {
    'model__max_depth': [5,10,15,20,25],
    'model__min_samples_leaf': [60,70,80,90,100]
}

CV_dt = GridSearchCV(
    model,
    param_grid,
    scoring='f1_weighted',
    n_jobs= -1,
    cv=5
)
                  
CV_dt.fit(X_train, y_train)  
print(CV_dt.best_params_)    
print(CV_dt.best_score_)

{'model__max_depth': 20, 'model__min_samples_leaf': 80}
0.4679232692335198
CPU times: user 2.24 s, sys: 274 ms, total: 2.51 s
Wall time: 25.2 s


### Logistic Regression model

In [27]:
%%time

#from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

model = lr

param_grid = {'model__solver':['newton-cg', 'lbfgs', 'sag', 'saga'],
              'model__penalty':['l2','none'],
              'model__C': [10,1,0.1]
}

CV_lr = GridSearchCV(
    model,
    param_grid,
    scoring='f1_weighted',
    n_jobs= -1,
    cv=5
)
                  
CV_lr.fit(X_train, y_train)  
print(CV_lr.best_params_)    
print(CV_lr.best_score_)

{'model__C': 10, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
0.4730942802123961
CPU times: user 2.72 s, sys: 311 ms, total: 3.03 s
Wall time: 9min 56s


### Random Forest model

In [28]:
%%time

model = rfc

param_grid = {
    'model__max_depth': [100,120,140,160,180,200],
    'model__min_samples_leaf': [0,1,2,3]
}

CV_rf = GridSearchCV(
    model,
    param_grid,
    scoring='f1_weighted',
    n_jobs= -1,
    cv=5
)
                  
CV_rf.fit(X_train, y_train)  
print(CV_rf.best_params_)    
print(CV_rf.best_score_)

{'model__max_depth': 160, 'model__min_samples_leaf': 1}
0.4687589143071154
CPU times: user 38.3 s, sys: 646 ms, total: 39 s
Wall time: 40min 35s


# Best model

In [43]:
# Let's make a summary of the results per model, before and after
# parameters tuning

final_models_evaluation = models_evaluation.copy()

final_models_evaluation.drop(['Weighted F1 std - no tuning', 'Weighted F1 scores - no tuning'],
                             axis=1, inplace=True)

final_models_evaluation.drop('dumb',inplace=True)

final_models_evaluation.rename(columns=
    {'Weighted F1 mean - no tuning':'Weighted F1 mean - before tuning'},
           inplace=True)

list_results_after_tunning = [CV_kn.best_score_, CV_linear_svc.best_score_, 
            CV_dt.best_score_, CV_lr.best_score_, CV_rf.best_score_]

list_results_after_tunning_rounded = []

for result in list_results_after_tunning:
    result = '{:.4f}'.format(result)
    list_results_after_tunning_rounded.append(result)

final_models_evaluation['Weighted F1 mean - after tuning'] = list_results_after_tunning_rounded

final_models_evaluation['Difference before & after'] = final_models_evaluation.apply(
lambda row: float(row['Weighted F1 mean - after tuning']) - float(row['Weighted F1 mean - before tuning']),
axis = 1)

final_models_evaluation['Ranking'] = final_models_evaluation['Weighted F1 mean - after tuning'].rank(
ascending=False)

final_models_evaluation['Ranking'] = final_models_evaluation['Ranking'].apply(lambda x: int(x))

final_models_evaluation

,Weighted F1 mean - before tuning,Weighted F1 mean - after tuning,Difference before & after,Ranking
kn,0.4401,0.4645,0.0244,5
linear_svc,0.4661,0.4678,0.0017,4
dt,0.4334,0.4679,0.0345,3
lr,0.4716,0.4731,0.0015,1
rfc,0.4625,0.4688,0.0063,2


The best model is the linear regression, with the parameters ```C = 10```, ```penalty = l2``` and ```solver = lbfgs```. As we can see in the [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), the logisitc regression classifier uses by default ```C = 1```, ```penalty = l2``` and ```solver = lbfgs```, so we only need to set the parameter ```C``` to ```10```.